 Tutaj zdefiniuj paramatry połączenia do API

In [2]:
import requests
import pandas as pd

# przygotuj adres URL API oraz token dostępowy
url = "https://api-datalab.coderslab.com/api/v2/"
api_token = "WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr"


 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [19]:
# wczytaj plik airports.csv i wyodrębnij identyfikatory lotnisk
airports = pd.read_csv(r"C:\Users\admin\Desktop\Koncowego_sprawdzony\data\airports.csv")
airports.head(10)

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638
5,14150
6,15323
7,14814
8,12007
9,11337


 Tutaj pobierz dane z endpoint'u `airport`

In [27]:
# nagłówek z tokenem 
headers = {'Authorization': 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'}

results = []
# iteracja po każdym lotnisku z listy
for airport_id in airports['origin_airport_id']:
    # adres endpointu airport
    url = 'https://api-datalab.coderslab.com/api/v2/airport/{}'.format(airport_id)
   
    # wykonanie zapytania do endpointu airport
    response = requests.get(url, headers=headers)

    # jeśli zapytanie zakończyło się sukcesem, dodaj wynik do listy
    if response.status_code == 200:
        result = response.json()
        results.append(result)

# konwersja wyników na ramkę danych i zapis do csv
airport_df = pd.DataFrame.from_records(results)
# airport_df.head(10)

    
 

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [30]:
airport_df = pd.DataFrame.from_records(results)
airport_df.to_csv(r'C:\Users\admin\Desktop\Koncowego_sprawdzony\data\raw\airport_data.csv', index=False)

In [4]:
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
import time
# nagłówek z tokenem
headers = {'Authorization': 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'}

# lista z danymi dotyczącymi zarejestrowanej pogody na lotniskach
weather_data = []

api_url_weather = 'https://api-datalab.coderslab.com/api/v2/airportWeather'
# data początkowa
start_date = pd.to_datetime('2019-01-01')
# data końcowa
end_date = pd.to_datetime('2020-03-31')
# iteracja po każdym identyfikatorze lotniska

#list for data from while loop
weather_data =[]

date = start_date
while date <= end_date:
    date_str = date.strftime("%Y-%m")
    response = requests.get(api_url_weather, headers=headers, params={"date": date_str})
    data = response.json()
    weather_data.extend(data)
    print(f'Downloading for date: {date_str} ...')
    date +=relativedelta(months=1)
    
    time.sleep(0.06)    

airport_weather_df = pd.json_normalize(weather_data)
print(airport_weather_df)


       WT18      STATION                                               NAME  \
0      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
1      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
2      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
3      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
4      None  USW00013874  ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...   
...     ...          ...                                                ...   
46221  None  USW00014762             PITTSBURGH ALLEGHENY CO AIRPORT, PA US   
46222  None  USW00014762             PITTSBURGH ALLEGHENY CO AIRPORT, PA US   
46223  None  USW00014762             PITTSBURGH ALLEGHENY CO AIRPORT, PA US   
46224  None  USW00014762             PITTSBURGH ALLEGHENY CO AIRPORT, PA US   
46225  None  USW00014762             PITTSBURGH ALLEGHENY CO AIRPORT, PA US   

             DATE   AWND  PRCP  SNOW  SNWD  TAVG  T

In [6]:
# konwersja wyników na ramkę danych i zapis do csv
airport_weather_df.to_csv(r'C:\Users\admin\Desktop\Koncowego_sprawdzony\data\raw\airport_weather.csv', index=False)

In [7]:
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
import time
# nagłówek z tokenem
headers = {'Authorization': 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'}

api_url_aircraft = 'https://api-datalab.coderslab.com/api/v2/aircraft'


response = requests.get(api_url_aircraft, headers=headers)
data = response.json()
time.sleep(0.06)    

aircraft_df = pd.json_normalize(data)
print(aircraft_df)

      MANUFACTURE_YEAR TAIL_NUM  NUMBER_OF_SEATS
0                 1944   N54514              0.0
1                 1945   N1651M              0.0
2                 1953   N100CE              0.0
3                 1953   N141FL              0.0
4                 1953   N151FL              0.0
...                ...      ...              ...
7378              2019   N14011            337.0
7379              2019   N16008            337.0
7380              2019   N16009            337.0
7381              2019   N2250U            276.0
7382              2019   N2749U            276.0

[7383 rows x 3 columns]


In [9]:
aircraft_df.to_csv(r'C:\Users\admin\Desktop\Koncowego_sprawdzony\data\raw\aircraft.csv', index=False)

In [55]:
import time
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
from requests import Session

api_url_flight = 'https://api-datalab.coderslab.com/api/v2/flight'

headers = {'Authorization': 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'}

airports_ids = airports['origin_airport_id'].tolist()

all_results = []
# data początkowa
start_date = pd.to_datetime('2019-01-01')
# data końcowa
end_date = pd.to_datetime('2020-03-31')

session = Session()
# petla po wszystkich lotniskach
for airport_id in airports_ids:
    params = {
        "airportId": airport_id,
        "date": start_date.strftime('%Y-%m-%d')
    }
    while params["date"] < end_date.strftime('%Y-%m-%d'):
        try:
            response = session.get(api_url_flight, headers=headers, params=params)
            if response.status_code == 200:
                results = json.loads(response.content)
                if results:
                    all_results.extend(results)
                else:
                    print(f'No data for airport: {airport_id} on date: {params["date"]}')
            elif response.status_code == 204:
                print(f'No data for airport: {airport_id} on date: {params["date"]}')
            else:
                print(f'Error: {response.status_code}  {response.content}')
            params['date'] = (datetime.strptime(params['date'], '%Y-%m-%d') + relativedelta(months=1)).strftime('%Y-%m-%d')
        except requests.exceptions.ConnectionError:
            print("Connection failed")
            time.sleep(10)
        time.sleep(0.06)
    print(f'Data for airport: {airport_id} loaded')
flight_df = pd.json_normalize(all_results)
flight_df


No data for airport: 10874 on date: 2019-01-01
No data for airport: 10874 on date: 2019-02-01
No data for airport: 10874 on date: 2019-03-01
No data for airport: 10874 on date: 2019-04-01
No data for airport: 10874 on date: 2019-05-01
No data for airport: 10874 on date: 2019-06-01
No data for airport: 10874 on date: 2019-07-01
No data for airport: 10874 on date: 2019-08-01
No data for airport: 10874 on date: 2019-09-01
No data for airport: 10874 on date: 2019-10-01
No data for airport: 10874 on date: 2019-11-01
No data for airport: 10874 on date: 2019-12-01
No data for airport: 10874 on date: 2020-01-01
No data for airport: 10874 on date: 2020-02-01
No data for airport: 10874 on date: 2020-03-01
Data for airport: 10874 loaded
No data for airport: 11233 on date: 2019-01-01
No data for airport: 11233 on date: 2019-02-01
No data for airport: 11233 on date: 2019-03-01
No data for airport: 11233 on date: 2019-04-01
No data for airport: 11233 on date: 2019-05-01
No data for airport: 11233 on

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,YEAR,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,210,205.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,210,220.0,1199,5,2019,4.0,0.0,10.0,0.0,3.0
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,120,124.0,581,3,2019,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386115,3,26,4,DL,N350DN,1982,13303,12953,1956,1944.0,...,183,169.0,1096,5,2020,NaN,NaN,NaN,NaN,NaN
1386116,3,26,4,DL,N908DE,1987,13303,10397,1120,1117.0,...,121,109.0,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386117,3,26,4,DL,NaN,1998,13303,10397,1817,NaN,...,125,NaN,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386118,3,26,4,DL,N352NW,2025,13303,10397,1937,1928.0,...,123,107.0,594,3,2020,NaN,NaN,NaN,NaN,NaN


In [57]:

flight_df.to_csv(r'C:\Users\admin\Desktop\Koncowego_sprawdzony\data\raw\flight.csv', index=False)